In [4]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Define a custom PyFunc model
class CustomModel(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None
    
    def fit(self, X, y):
        # Implement your custom model fitting logic here
        # For this example, we'll use a simple linear regression
        self.model = np.linalg.lstsq(X, y, rcond=None)[0]
    
    def predict(self, context, model_input):
        return np.dot(model_input, self.model)

# Set up MLflow experiment
mlflow.set_experiment("CustomPyFuncExperiment")

# Generate sample data
X = np.random.rand(100, 3)
y = np.sum(X, axis=1) + np.random.normal(0, 0.1, 100)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Start MLflow run
with mlflow.start_run():
    # Create and train the custom model
    custom_model = CustomModel()
    custom_model.fit(X_train_scaled, y_train)
    
    # Log model parameters (if any)
    mlflow.log_param("num_features", X.shape[1])
    
    # Make predictions
    y_pred = custom_model.predict(None, X_test_scaled)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)
    
    # Log the custom model as a PyFunc
    mlflow.pyfunc.log_model("custom_model", python_model=custom_model)
    
    run_id = mlflow.active_run().info.run_id
    
    print(f"Model logged with run_id: {mlflow.active_run().info.run_id}")
    print(f"MSE: {mse:.4f}")
    print(f"R2 Score: {r2:.4f}")



2024/09/30 10:17:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model logged with run_id: f940d5cc92a3408abdbd67a04592f534
MSE: 1.9571
R2 Score: -6.6511


In [5]:
# Load the logged model
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/custom_model")

# Make predictions using the loaded model
new_data = np.random.rand(10, 3)
new_data_scaled = scaler.transform(new_data)
predictions = loaded_model.predict(new_data_scaled)

print("Predictions from loaded model:")
print(predictions)


Predictions from loaded model:
[-0.24220425  0.28896695  0.26188466 -0.25961663  0.02547142  0.53590976
 -0.00808452 -0.45540666 -0.112865    0.61923423]


In [6]:

# Load the model from the experiment
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/custom_model")

# Register the loaded model in the Model Registry
model_name = "CustomPyFuncModel_Example"
model_version = mlflow.register_model(f"runs:/{run_id}/custom_model", model_name)

print(f"Model registered as: {model_name} with version number {model_version.version}")

# Make predictions using the loaded model
new_data = np.random.rand(10, 3)
new_data_scaled = scaler.transform(new_data)
predictions = loaded_model.predict(new_data_scaled)

print("Predictions from loaded model:")
print(predictions)

Model registered as: CustomPyFuncModel_Example version 1
Predictions from loaded model:
[ 0.23004414 -0.2793218   0.58607866  0.30976924 -0.00996094 -0.45184482
  0.55206215  0.37900833 -0.10939631  0.34033454]


Successfully registered model 'CustomPyFuncModel_Example'.
Created version '1' of model 'CustomPyFuncModel_Example'.
